In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import time
import datetime

# Подход 1: градиентный бустинг "в лоб"

### 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [99]:
data_train = pd.read_csv('features.csv', index_col='match_id')
data_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [100]:
drop_features = ['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant',
                 'barracks_status_dire']
data_train_dropped = data_train.drop(drop_features, axis=1)

In [101]:
data_train_dropped.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


### 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

##### Определяем максимальное количество элементов в столбце

In [102]:
empty_val = data_train_dropped.count()

In [103]:
max_val = empty_val[0]
for i in empty_val:
    if i>max_val:
        max_val = 1
print(max_val)

97230


##### Ищем Id и названия колонок, количество значений в которых меньше максимального значения

In [104]:
columns_empty = []
for name, amount in enumerate(data_train_dropped.count()):
    if amount<max_val:
        columns_empty.append(name)
col_empty_names = []
for i, value in enumerate(data_train_dropped):
    for k in columns_empty:
        if i == k:
            col_empty_names.append(value)
print(col_empty_names)

['first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time']


###### Список столбцов с пропущенными значениями в массиве выше(col_empty_names). Для признаков first_blood_time, first_blood_team, first_blood_player1, first_blood_player2 значения пропущены в части объектов, потому что события соответствующие признакам случились позже, чем через 5 минут после начала матча 

### 3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [105]:
data_train_dropped = data_train_dropped.fillna(0)

### 4. Какой столбец содержит целевую переменную? Запишите его название.

##### radiant_win

### 5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [106]:
y_train = data_train['radiant_win']
X_train = data_train_dropped

In [106]:
kf = KFold(n_splits=5, shuffle=True, random_state=241)
start_time = datetime.datetime.now()
time.sleep(3)
grid = {'n_estimators': [10,20,30]}
GBC = GradientBoostingClassifier(random_state=241)
gs = GridSearchCV(GBC, grid, scoring='roc_auc', cv=kf)
gs.fit(X_train, y_train)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:03:53.950836


In [107]:
gs.best_params_

{'n_estimators': 30}

In [108]:
gs.best_score_

0.6894962060591201

##### Максимальное значение метрики достигнуто при количестве деревьев 30. Это максималньое значение в массиве количества деревьев и говорит о том, что качество продолжит расти с увеличением количества деревьев. Для наглядности возьмем другие значения количества деревьев

In [107]:
kf = KFold(n_splits=5, shuffle=True, random_state=241)
start_time = datetime.datetime.now()
time.sleep(3)
grid = {'n_estimators': range(30, 50, 10)}
GBC = GradientBoostingClassifier(random_state=241)
gs = GridSearchCV(GBC, grid, scoring='roc_auc', cv=kf)
gs.fit(X_train, y_train)
print ('Time elapsed:', datetime.datetime.now() - start_time)

KeyboardInterrupt: 

In [50]:
gs.best_params_

{'n_estimators': 70}

In [51]:
gs.best_score_

0.7020316980434621

##### Видим, что качество увеличивается с увеличением количества деревьев

# Подход 2: логистическая регрессия

### 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [108]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [109]:
kf = KFold(n_splits=5, shuffle=True, random_state=241)
start_time = datetime.datetime.now()
time.sleep(3)
grid = {'C': np.power(10.0, np.arange(-5, 5))}
clf = LogisticRegression(penalty = 'l2',random_state=241, solver = 'lbfgs')
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv=kf)
gs.fit(X_train_scaled, y_train)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:00:50.469647


In [110]:
gs.best_params_

{'C': 0.01}

In [111]:
gs.best_score_

0.7163420915422015

##### Лучшее качество 0.71 - немного выше, чем у градиентного бустинга при 70 деверьях. Но, в бустинге я смогу увеличить количество деревьев и увеличится качество

### 2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [112]:
categorial = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero',
             'd3_hero', 'd4_hero', 'd5_hero']
X_train = X_train.drop(categorial, axis = 1)
X_train_scaled = scaler.fit_transform(X_train)

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [113]:
start_time = datetime.datetime.now()
time.sleep(3)
grid = {'C': np.power(10.0, np.arange(-5, 5))}
kf = KFold(n_splits=5, shuffle=True, random_state=241)
clf = LogisticRegression(penalty = 'l2',random_state=241, solver = 'lbfgs')
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv=kf)
gs.fit(X_train_scaled, y_train)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:00:50.035967


In [114]:
gs.best_params_

{'C': 0.01}

In [115]:
gs.best_score_

0.7164004932889205

##### Качество очень мало изменилось при извлечении этих признаков и это логично, так как они имели малый вес в алгоритме из-за того, чтобы были цыфрами, зависимости между которыми было невозможно восстановить

### 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [116]:
len(data_train['d1_hero'].unique())

108

##### Существует 108 различный идентификаторов

### 4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [117]:
categ = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero',
             'd3_hero', 'd4_hero', 'd5_hero']
data = data_train[categ]
X_pick = np.zeros((data.shape[0], 108))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-5] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-5] = -1

In [118]:
X_pick.shape

(97230, 108)

In [119]:
X_train = np.hstack((X_train_scaled, X_pick))

In [120]:
X_train.shape

(97230, 199)

### Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [121]:
start_time = datetime.datetime.now()
time.sleep(3)
grid = {'C': np.power(10.0, np.arange(-5, 5))}
kf = KFold(n_splits=5, shuffle=True, random_state=241)
clf = LogisticRegression(penalty = 'l2',random_state=241, solver = 'lbfgs', max_iter = 500)
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv=kf)
gs.fit(X_train, y_train)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:02:12.955041


In [122]:
gs.best_score_

0.7499077532806516

In [123]:
gs.best_params_

{'C': 0.1}

#####  Качество улучшилось, потому что добавились признаки, которые имеют высокий вес при предсказании результата

### Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [124]:
X_test = pd.read_csv('features_test.csv', index_col='match_id')

##### Сделаем нужные преобразования выборки для того, чтобы мы могли работать с созданным алгоритмом

In [125]:
X_test = X_test.fillna(0)

In [126]:
categorial = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero',
             'd3_hero', 'd4_hero', 'd5_hero']
X_test = X_test.drop(categorial, axis = 1)

In [127]:
X_test_scaled = scaler.fit_transform(X_test)

C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ivan1\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [128]:
categ = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero',
             'd3_hero', 'd4_hero', 'd5_hero']
data = data_test[categ]
X_pick = np.zeros((data.shape[0], 108))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-5] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-5] = -1

In [129]:
X_test = np.hstack((X_test_scaled, X_pick))

In [130]:
X_test.shape

(17177, 199)

In [131]:
test_estim = gs.predict_proba(X_test)

In [132]:
min(test_estim[:, 1])

0.00619020053493267

In [133]:
max(test_estim[:, 1])

0.9975308275023299

##### Сделаем оценку леса, для того чтобы понять как новые признаки влияют на качество алгоритма

In [88]:
kf = KFold(n_splits=5, shuffle=True, random_state=241)
start_time = datetime.datetime.now()
time.sleep(3)
grid = {'n_estimators': [10,20,30]}
GBC = GradientBoostingClassifier(random_state=241)
gs = GridSearchCV(GBC, grid, scoring='roc_auc', cv=kf)
gs.fit(X_train, y_train)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:08:17.023593


In [89]:
gs.best_score_

0.689708065181296

In [90]:
gs.best_params_

{'n_estimators': 30}